In [ ]:
require 'sqlite3'

db = SQLite3::Database.new 'transactions.db'

In [ ]:
db.execute <<-SQL
DROP TABLE bank_transactions
SQL

db.execute <<-SQL
CREATE TABLE bank_transactions (
  date varchar(30),
  to_acct varchar(255),
  from_acct varchar(255),
  /* Stored as USD cents */
  amount int,
  memo text
)
SQL

In [ ]:
require 'watir'
require 'webdrivers'
require 'date'

browser = nil
begin
  browser = Watir::Browser.new :chrome,
                              switches: %w[ --headless
                                            --disable-gpu --no-sandbox
                                            --disable-prompt-on-repost
                                            --windows-size=3200x1800]

  browser.goto 'https://secure.ally.com/'
  browser.text_field(id: 'login-username').set ENV['ALLY_USER']
  browser.text_field(id: 'login-password').set ENV['ALLY_PASSWORD']
  browser.button(type: 'submit').click

  Watir::Wait.until { browser.title.include? 'Dashboard' }

  # Navigate to transfers page
  browser.link(text: 'Transfers').click
  Watir::Wait.until { browser.title.include? 'Make Transfer' }

  # Open activity tab
  browser.link(text: 'Activity').click
  Watir::Wait.until { browser.title.include? 'Activity' }

  history_table = browser.p(text: 'History').next_sibling

  rows = history_table.tbody.trs(id: /.*_ActivityTr/)
  
  already_exist_tries = 3

  rows.each do |row|
    date = row.td(name: 'activityDeliveryDate').text
    date = Date.strptime(date, '%b %d, %Y')
    accounts = row.spans(class: 'activity-account-link')
    to = accounts[0].text[/To ([^\*]*) [\*\d]+/,1]
    from = accounts[1].text[/From ([^\*]*) [\*\d]+/,1]
    amount = row.tds[2].text.gsub(/[^\d]/, '').to_i * 10
    
    # Expand panel to get details
    expand_link = row.tds[1].a
    expand_link.click
    
    Watir::Wait.until { row.next_sibling.text.include? 'Transfer Status' }
    
    note = ''
    if row.next_sibling.span(class: 'memo-multiline-block').exist?
      note = row.next_sibling.span(class: 'memo-multiline-block').text
    end
    
    values = [date.to_s, to, from, amount, note]
    
    already_exists = false
    db.execute( "select count(*) as count from bank_transactions where date = ? AND to_acct = ? AND from_acct = ? AND amount = ? AND memo = ?", values) do |count|
      already_exists = count[0] > 0
    end
    
    if already_exists
      already_exist_tries -= 1
    else
      p values
      db.execute "insert into bank_transactions values (?, ?, ?, ?, ?)", values
    end
    
    break if already_exist_tries == 0
  end


rescue => ex
    p ex
    browser.screenshot.save "error.png"
ensure
  browser.quit if browser
end

nil

In [ ]:
  db.execute( "select * from bank_transactions order by date desc") do |row|
    p row
  end